# 1. Install AG2 + Materials Project dependencies

In [1]:
!pip install -U "ag2[openai]" autogen -q

import sys
!{sys.executable} -m pip install pymatgen mp-api numpy cython ipywidgets jupyterlab_widgets -q

print("✓ AG2/autogen + Pymatgen + MP-API installed.")

✓ AG2/autogen + Pymatgen + MP-API installed.


# 2. Imports + LLM Configuration


In [29]:
import os
import json
import random
from datetime import datetime, timedelta
from typing import Annotated, Any, Literal, Union, List, Optional, Tuple

# AG2 core imports
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import ReplyResult
from autogen.agentchat.group import (
    ContextVariables,
    AgentTarget, AgentNameTarget, StayTarget,
    OnCondition, StringLLMCondition,
    OnContextCondition, ExpressionContextCondition, ContextExpression,
    RevertToUserTarget, TerminateTarget,
)
from autogen.agentchat.groupchat import GroupChat, GroupChatManager

# Tools + External APIs
from mp_api.client import MPRester

# Coding tools
from autogen.coding.local_commandline_code_executor import LocalCommandLineCodeExecutor
from autogen.coding.base import CodeBlock
from pydantic import BaseModel, Field

# LLM configuration
llm_config = LLMConfig(
    config_list=[
        {
            "model": "gpt-4o",
            "api_key": os.environ["OPENAI_API_KEY"],
        }
    ],
)

print("✓ Imports loaded and LLM configured.")



✓ Imports loaded and LLM configured.


In [3]:
import autogen
print(autogen.__version__)


0.10.2


# 3. User query + Context Variables

In [30]:
from autogen.agentchat.group import ContextVariables

# Context variables (empty initial state — A will fill them)
context_variables = ContextVariables(
    data={
        "task_started": False,
        "query": None,                   # will be filled by Agent A (explain_query_tool)
        "query_explanation": None,       # filled by Agent A
        "search_criteria": None,         # filled by Agent B
        "fields": None,                  # filled by Agent B
        "sample_number": None,           # filled by Agent B
        "mp_results": None,              # filled by Agent B
        "final_conclusion": None,        # filled by Agent C
        "next_agent": None,
    }
)

print("✓ ContextVariables correctly initialized for AG2 multi-agent pipeline.")




✓ ContextVariables correctly initialized for AG2 multi-agent pipeline.


# 4. TOOLS (A → B → C → D)

In [31]:
# Tool A — explain_query

def explain_query_tool(
    query: Annotated[str, "initial query AS provided by the user without modification."],
    query_explanation: Annotated[str, "An explanation of the query, including a breakdown of key and secondary terms, 2-3 sentences."],
    context_variables: ContextVariables,
) -> ReplyResult:
    """Explain the posed query. What does it ask? Include a breakdown of the key and secondary terms in the query."""

    # Basic validation
    try:
        if not isinstance(query, str):
            raise ValueError("The 'query' parameter must be a string.")
        if len(query.strip()) == 0:
            raise ValueError("Query cannot be empty.")
        # (opcional, pero coherente) también podríamos validar query_explanation:
        # if not isinstance(query_explanation, str) or len(query_explanation.strip()) == 0:
        #     raise ValueError("query_explanation must be a non-empty string.")
    except ValueError as e:
        target_agent = AgentNameTarget("AgentA_Explainer")
        return ReplyResult(
            message=f"Error in explain_query_tool: {e}",
            target=target_agent,
            context_variables=context_variables,
        )

    # Build explanation
    explanation = (
        f"Explanation of user query: '{query}'\n\n"
        f"- Query explanation: {query_explanation}"
    )

    # Update context
    context_variables["task_started"] = True
    context_variables["query"] = query                         
    context_variables["query_explanation"] = query_explanation
    context_variables["explained_terms"] = query_explanation
    context_variables["next_agent"] = "AgentB_MaterialsRetriever"

    # Set next agent
    target_agent = AgentNameTarget("AgentB_MaterialsRetriever")

    return ReplyResult(
        message=explanation,
        target=target_agent,
        context_variables=context_variables,
    )


# Tool B — download_materials_structures_properties_from_mp

def material_retiever(
    search_criteria: Annotated[dict, "Filtering conditions used to select candidate materials."],
    fields: Annotated[List[str], "List of metadata fields to retrieve for each material."],
    sample_number: Annotated[int, "Number of materials to randomly sample and download from the filtered set."],
    context_variables: ContextVariables,
) -> ReplyResult:
    """Retrieve materials from Materials Project using search_criteria, fields, and sample_number."""

    try:
        if not isinstance(search_criteria, dict):
            raise ValueError("search_criteria must be a dictionary.")
        if not isinstance(fields, list) or not all(isinstance(f, str) for f in fields):
            raise ValueError("fields must be a list of strings.")
        if not isinstance(sample_number, int) or sample_number <= 0:
            raise ValueError("sample_number must be a positive integer.")
    except ValueError as e:
        target_agent = AgentNameTarget("AgentB_MaterialsRetriever")
        return ReplyResult(
            message=f"Error in material_retiever: {e}",
            target=target_agent,
            context_variables=context_variables,
        )

    alias_map = {
        "num_sites": "nsites",
        "k_voigt": "bulk_modulus",
        "g_voigt": "shear_modulus",
    }

    def normalize_range(value):
        if isinstance(value, dict):
            lower = None
            upper = None
            if "$gt" in value:
                lower = value["$gt"]
            if "$gte" in value:
                lower = value["$gte"]
            if "$lt" in value:
                upper = value["$lt"]
            if "$lte" in value:
                upper = value["$lte"]
            return (lower, upper)
        return value

    normalized_criteria = {}
    for key, value in search_criteria.items():
        mapped_key = alias_map.get(key, key)
        normalized_criteria[mapped_key] = normalize_range(value)

    api_key = os.getenv("MP_API_KEY")
    try:
        if not api_key:
            raise ValueError("Missing MP_API_KEY environment variable.")
    except ValueError as e:
        target_agent = AgentNameTarget("AgentB_MaterialsRetriever")
        return ReplyResult(
            message=f"Error in material_retiever: {e}",
            target=target_agent,
            context_variables=context_variables,
        )

    try:
        with MPRester(api_key) as mpr:
            all_results = list(
                mpr.materials.summary.search(
                    fields=fields,
                    **normalized_criteria,
                )
            )
    except Exception as e:
        context_variables["mp_results"] = []
        context_variables["search_criteria"] = search_criteria
        context_variables["fields"] = fields
        context_variables["sample_number"] = sample_number
        context_variables["next_agent"] = "AgentB_MaterialsRetriever"
        return ReplyResult(
            message=f"Error in material_retiever while querying Materials Project: {repr(e)}",
            target=AgentNameTarget("AgentB_MaterialsRetriever"),
            context_variables=context_variables,
        )

    if not all_results:
        message = (
            "No materials found for the given search_criteria. "
            "This may be due to a typo/inconsistency or because no materials satisfy the constraints. "
            "Please adapt the filters while keeping the main constraints in place."
        )
        context_variables["mp_results"] = []
        context_variables["search_criteria"] = search_criteria
        context_variables["fields"] = fields
        context_variables["sample_number"] = sample_number
        context_variables["next_agent"] = "AgentB_MaterialsRetriever"

        return ReplyResult(
            message=message,
            target=AgentNameTarget("AgentB_MaterialsRetriever"),
            context_variables=context_variables,
        )

    if sample_number < len(all_results):
        results = random.sample(all_results, sample_number)
    else:
        results = all_results

    context_variables["search_criteria"] = search_criteria
    context_variables["fields"] = fields
    context_variables["sample_number"] = sample_number
    context_variables["mp_results"] = results
    context_variables["next_agent"] = "AgentC_Analyzer"

    message = (
        f"Retrieved {len(results)} materials from Materials Project "
        f"with search_criteria={search_criteria}. "
        "Results stored in context_variables['mp_results'] for further analysis."
    )

    target_agent = AgentNameTarget("AgentC_Analyzer")

    return ReplyResult(
        message=message,
        target=target_agent,
        context_variables=context_variables,
    )


# Tool C — final_conclusion_tool (Analyzer)

def final_conclusion_tool(
    context_variables: ContextVariables,
) -> ReplyResult:
    """Build a structured analysis of the retrieved materials and store it in context_variables['final_conclusion']."""

    results = context_variables.get("mp_results", None)

    # No results available
    if not results:
        target_agent = AgentNameTarget("AgentC_Analyzer")
        return ReplyResult(
            message="No materials available in context_variables['mp_results'] to analyze.",
            target=target_agent,
            context_variables=context_variables,
        )

    structured_output = []

    for entry in results:
        def get(k):
            if isinstance(entry, dict):
                return entry.get(k)
            return getattr(entry, k, None)

        material_block = {
            "Material": {
                "ID": get("material_id"),
                "Formula": get("formula_pretty") or get("pretty_formula"),
                "Bandgap": get("band_gap"),
                "Density": get("density"),
                "Volume": get("volume"),
                "Energy_above_hull": get("energy_above_hull") or get("e_above_hull"),
            },
            "Usage": {
                "Applications": "Potential applications based on retrieved properties.",
                "Industry_relevance": "Why this material could matter for industry.",
            },
            "Sustainability": {
                "Abundance": "Placeholder for abundance / criticality analysis.",
                "Environmental_impact": "Placeholder evaluation of toxicity / extraction cost.",
                "Recyclability": "Placeholder for recyclability commentary.",
            },
            "Weaknesses": {
                "Structural_limitations": "Placeholder for mechanical or thermal limitations.",
                "Economic_limitations": "Placeholder for cost or manufacturability issues.",
            },
        }

        structured_output.append(material_block)

    # Save structured result
    context_variables["final_conclusion"] = structured_output
    context_variables["next_agent"] = "Human"

    import json
    header = f"Produced a structured analysis for {len(structured_output)} materials.\n"
    analysis_text = json.dumps(structured_output, indent=2, default=str)

    message = header + "\nStructured analysis (JSON):\n" + analysis_text

    target_agent = AgentNameTarget("Human")

    return ReplyResult(
        message=message,
        target=target_agent,
        context_variables=context_variables,
    )

# Tool D — python_coder

project_folder = os.path.abspath("ag2_project")
os.makedirs(project_folder, exist_ok=True)
os.environ["PROJECT_FOLDER"] = project_folder


class PythonCode(BaseModel):
    code: str = Field(..., description="Full python code executed by the coder agent")


def python_coder_tool(
    code: Annotated[str, "A single block of Python code to execute."],
    file_name: Annotated[str, "Name of the code file to store the executed script, e.g., 'script.py'"],
    context_variables: ContextVariables,
) -> ReplyResult:
    """Execute Python code, save it to ag2_project/<file_name>, update a persistent JSON context, and return the execution output."""

    try:
        if not isinstance(code, str) or len(code.strip()) == 0:
            raise ValueError("The 'code' parameter must be a non-empty string.")
        if not isinstance(file_name, str) or len(file_name.strip()) == 0:
            raise ValueError("The 'file_name' parameter must be a non-empty string.")
    except ValueError as e:
        return ReplyResult(
            message=f"Error in python_coder_tool: {e}",
            target=AgentNameTarget("AgentD_Coder"),
            context_variables=context_variables,
        )

    full_code = "#!/usr/bin/env python3\n" + code
    python_code_model = PythonCode(code=full_code)

    # Execute code
    executor = LocalCommandLineCodeExecutor(timeout=60000)
    code_block = CodeBlock(language="python", code=full_code)
    try:
        result = executor.execute_code_blocks([code_block])
    except Exception as e:
        message = (
            "Code execution failed due to an internal executor error:\n"
            f"{e}\n\n"
            "Correct the code or environment and call python_coder_tool again."
        )
        return ReplyResult(
            message=message,
            target=AgentNameTarget("AgentD_Coder"),
            context_variables=context_variables,
        )

    exit_code = result.exit_code
    output = result.output

    # Persistent context file
    context_path = os.path.join(project_folder, "context_variables_data.json")

    try:
        with open(context_path, "r") as f:
            context_file_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        context_file_data = {
            "execution_results": {},
            "execution_history": [],
            "execution_notes": [],
        }

    # Update context based on exit_code
    if exit_code != 0:
        context_file_data["code_error"] = python_code_model.model_dump()
    else:
        if "code" not in context_file_data or not isinstance(
            context_file_data.get("code", []), list
        ):
            context_file_data["code"] = []
        context_file_data["code"].append(python_code_model.model_dump())
        context_file_data["code_error"] = ""

    # Store basic execution info
    context_file_data["execution_results"][file_name] = {
        "exit_code": exit_code,
        "output": output,
    }
    context_file_data["execution_history"].append(
        f"Executed {file_name} with exit_code={exit_code}"
    )
    context_file_data["execution_notes"].append(
        f"Executed {file_name}. Stored output and exit code in execution_results."
    )

    # Save executed code
    code_output_path = os.path.join(project_folder, file_name)
    os.makedirs(os.path.dirname(code_output_path), exist_ok=True)
    with open(code_output_path, "w") as f:
        f.write(full_code)

    # Remove temporary file created by executor (if any)
    try:
        if result.code_file and os.path.exists(result.code_file):
            os.remove(result.code_file)
    except Exception:
        pass

    # Save persistent JSON context file
    with open(context_path, "w") as f:
        json.dump(context_file_data, f, indent=2)

    # Also store the last run in the in-memory context_variables
    context_variables["last_executed_code"] = full_code
    context_variables["last_execution_output"] = output

    # Build return message
    if exit_code == 0:
        message = (
            "Code executed successfully with message:\n"
            f"{output}\n\n"
            "Check if the output contains any failure or error message. "
            "If yes, you must fix the error and try again via `python_coder_tool`."
        )
    else:
        message = (
            "Code execution failed with error message:\n"
            f"{output}\n\n"
            "Correct the code and call python_coder_tool again."
        )

    target_agent = AgentNameTarget("AgentD_Coder")

    return ReplyResult(
        message=message,
        target=target_agent,
        context_variables=context_variables,
    )


# 5. Manual tests for material_retiever


In [32]:
print("Running manual tests for material_retiever...\n")

from autogen.agentchat.group import ContextVariables

# Test 1 — Simple valid query
try:
    search_criteria_1 = {
        "band_gap": (3, 5),
        "energy_above_hull": (0, 0.1),
    }
    fields_1 = ["material_id", "formula_pretty", "band_gap", "energy_above_hull"]
    sample_number_1 = 5

    ctx_1 = ContextVariables(data={})
    result_1 = material_retiever(
        search_criteria=search_criteria_1,
        fields=fields_1,
        sample_number=sample_number_1,
        context_variables=ctx_1,
    )
    print("Test 1 completed. Message:")
    print(result_1.message)
except Exception as e:
    print("Test 1 failed with unexpected exception:", e)


# Test 2 — basic test with simple filters
try:
    search_criteria_2 = {
        "band_gap": (0, 1),
        "energy_above_hull": (0, 0.05),   
    }
    fields_2 = ["material_id", "band_gap", "energy_above_hull"]
    sample_number_2 = 3

    ctx_2 = ContextVariables(data={})
    result_2 = material_retiever(
        search_criteria=search_criteria_2,
        fields=fields_2,
        sample_number=sample_number_2,
        context_variables=ctx_2,
    )
    print("\nTest 2 completed. Message:")
    print(result_2.message)
except Exception as e:
    print("Test 2 raised exception:", e)



# Test 3 — Empty fields
try:
    search_criteria_3 = {
        "band_gap": (1, 3),
    }
    fields_3 = []
    sample_number_3 = 3

    ctx_3 = ContextVariables(data={})
    result_3 = material_retiever(
        search_criteria=search_criteria_3,
        fields=fields_3,
        sample_number=sample_number_3,
        context_variables=ctx_3,
    )
    print("\nTest 3 completed (empty fields). Message:")
    print(result_3.message)
except Exception as e:
    print("Test 3 raised exception (likely due to empty fields):", e)


# Test 4 — invalid sample_number
try:
    search_criteria_4 = {
        "band_gap": (0, 10),
    }
    fields_4 = ["material_id"]
    sample_number_4 = 0

    ctx_4 = ContextVariables(data={})
    result_4 = material_retiever(
        search_criteria=search_criteria_4,
        fields=fields_4,
        sample_number=sample_number_4,
        context_variables=ctx_4,
    )
    print("\nTest 4 completed (sample_number <= 0). Message:")
    print(result_4.message)
except Exception as e:
    print("Test 4 raised exception (expected due to validation):", e)


# Test 5 — Invalid key 
try:
    search_criteria_5 = {
        "invalid_property": (0, 1),
    }
    fields_5 = ["material_id"]
    sample_number_5 = 3

    ctx_5 = ContextVariables(data={})
    result_5 = material_retiever(
        search_criteria=search_criteria_5,
        fields=fields_5,
        sample_number=sample_number_5,
        context_variables=ctx_5,
    )
    print("\nTest 5 completed (invalid filter key). Message:")
    print(result_5.message)
except Exception as e:
    print("Test 5 raised exception (this will guide future validation):", e)


print("\nManual tool tests completed.")


Running manual tests for material_retiever...



Retrieving SummaryDoc documents:   0%|          | 0/14851 [00:00<?, ?it/s]

Test 1 completed. Message:
Retrieved 5 materials from Materials Project with search_criteria={'band_gap': (3, 5), 'energy_above_hull': (0, 0.1)}. Results stored in context_variables['mp_results'] for further analysis.


Retrieving SummaryDoc documents:   0%|          | 0/43649 [00:00<?, ?it/s]


Test 2 completed. Message:
Retrieved 3 materials from Materials Project with search_criteria={'band_gap': (0, 1), 'energy_above_hull': (0, 0.05)}. Results stored in context_variables['mp_results'] for further analysis.


Retrieving SummaryDoc documents:   0%|          | 0/34231 [00:00<?, ?it/s]

/opt/miniconda3/lib/python3.13/site-packages/mp_api/client/core/client.py:841: UserWarning: Use the 'fields' argument to select only fields of interest to speed up data retrieval for large queries. Choose from: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_fo


Test 3 completed (empty fields). Message:
Retrieved 3 materials from Materials Project with search_criteria={'band_gap': (1, 3)}. Results stored in context_variables['mp_results'] for further analysis.

Test 4 completed (sample_number <= 0). Message:
Error in material_retiever: sample_number must be a positive integer.

Test 5 completed (invalid filter key). Message:
Error in material_retiever while querying Materials Project: MPRestError('You have specified the following kwargs which are unknown to \x1b`search`\x1b, but may be known to \x1b`_search`\x1b\n    \x1binvalid_property\x1b\nPlease see the documentation:\n    \x1b`search`: https://materialsproject.github.io/api/_autosummary/mp_api.client.routes.materials.summary.SummaryRester.html#mp_api.client.routes.materials.summary.SummaryRester.search\x1b\n   \x1b`_search`: https://api.materialsproject.org/redoc#tag/Materials-Summary/operation/search_materials_summary__get\x1b')

Manual tool tests completed.


# 6. AGENTS (A → B → C → D + HUMAN)

In [33]:
# Agent A: Explainer

explainer_message = """
You are an explainer AI agent.

ROLE
- Interpret the user's materials-science question.
- Produce a short, structured scientific explanation.

TASKS
- Read the query as provided in the latest user message.
- Identify key scientific terms (e.g., band gap, symmetry, density, conductivity).
- Explain what the user is asking in clear, technically correct language.

TOOL USAGE
- You must always call explain_query_tool.
- Do not answer directly without using the tool.
"""

AgentA_Explainer = ConversableAgent(
    name="AgentA_Explainer",
    llm_config=llm_config,
    system_message=explainer_message,
    human_input_mode="NEVER",
    functions=[explain_query_tool],
    function_map={"explain_query_tool": explain_query_tool},
)


# Agent B: Materials Retriever 

retriever_message = """
You are a materials retriever AI agent.

ROLE
- Turn the explained query into a valid Materials Project search.

TASKS
- Read the explanation from context_variables["query_explanation"].
- Build search_criteria, fields and sample_number consistent with the user's query.
- Call the retriever tool to query Materials Project.
- Store results in context_variables["mp_results"] for the analyzer.

TOOL USAGE
- You must always call material_retiever.
- Do not write free-form analysis; leave that to the analyzer.
- Be careful with keys in search_criteria, for example:
  - Numeric filters: band_gap, energy_above_hull, k_voigt, g_voigt, num_sites
  - Composition filters: elements, chemsys, excluded_elements
"""

AgentB_MaterialsRetriever = ConversableAgent(
    name="AgentB_MaterialsRetriever",
    llm_config=llm_config,
    system_message=retriever_message,
    human_input_mode="NEVER",
    functions=[material_retiever],
    function_map={"material_retiever": material_retiever},
)


# Agent C: Analyzer

analyzer_message = """
You are an analyzer AI agent.

ROLE
- Analyze and organize the Materials Project results.

TASKS
- Read raw results from context_variables["mp_results"].
- Use the analysis tool to build a structured scientific summary.
- Focus on properties, applications, sustainability and limitations.

OUTPUT
- A structured analysis stored in context_variables["final_conclusion"].
- The message returned by the tool is what the Human will see.

TOOL USAGE
- You must always call final_conclusion_tool.
- Do not answer directly without using the tool.
"""

AgentC_Analyzer = ConversableAgent(
    name="AgentC_Analyzer",
    llm_config=llm_config,
    system_message=analyzer_message,
    human_input_mode="NEVER",
    functions=[final_conclusion_tool],
    function_map={"final_conclusion_tool": final_conclusion_tool},
)


# Agent D: Coding Agent

coder_message = """
You are a coder AI agent.

ROLE
- Generate and execute Python code when needed.

TASKS
- Write Python code for calculations, data handling or plots.
- Execute all code only via python_coder_tool.
- Save executed code under ag2_project/<file_name>.
- Use the tool output to decide the next coding step.

TOOL USAGE
- You must always call python_coder_tool.
- Do not execute code directly or reply with analysis only.
"""

AgentD_Coder = ConversableAgent(
    name="AgentD_Coder",
    llm_config=llm_config,
    system_message=coder_message,
    human_input_mode="NEVER",
    functions=[python_coder_tool],
    function_map={"python_coder_tool": python_coder_tool},
)


# Human Agent

Human = ConversableAgent(
    name="Human",
    llm_config=None,
    human_input_mode="ALWAYS",
)
print("✓ Agents created.")

✓ Agents created.


# 7. Pattern definition 


In [34]:
# Pattern: select next speaker based on last agent and context_variables

def select_next_speaker(last_speaker, groupchat):
    """Simple manual pattern using context_variables['next_agent'] when available."""
    cv = groupchat.context_variables
    agents_by_name = {a.name: a for a in groupchat.agents}

    # First turn or after Human -> Agent A
    if last_speaker is None or last_speaker.name == "Human":
        return agents_by_name.get("AgentA_Explainer", last_speaker)

    # If a tool set an explicit next agent
    next_name = cv.get("next_agent")
    if isinstance(next_name, str) and next_name in agents_by_name:
        return agents_by_name[next_name]

    # Default flow (A -> B -> C -> D -> Human)
    if last_speaker.name == "AgentA_Explainer":
        return agents_by_name.get("AgentB_MaterialsRetriever", last_speaker)
    if last_speaker.name == "AgentB_MaterialsRetriever":
        return agents_by_name.get("AgentC_Analyzer", last_speaker)
    if last_speaker.name == "AgentC_Analyzer":
        return agents_by_name.get("AgentD_Coder", last_speaker)
    if last_speaker.name == "AgentD_Coder":
        return agents_by_name.get("Human", last_speaker)

    # Fallback
    return agents_by_name.get("Human", last_speaker)
    
print("✓ Pattern defined.")     

✓ Pattern defined.


# 8. Group Chat

In [ ]:
# 8. GroupChat setup

groupchat = GroupChat(
    agents=[
        AgentA_Explainer,
        AgentB_MaterialsRetriever,
        AgentC_Analyzer,
        AgentD_Coder,
        Human,
    ],
    messages=[],
    max_round=20,
)

# Attach context variables
groupchat.context_variables = context_variables

manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
)

print("✓ GroupChat and Manager ready.")

# First user query (real interactive use)
initial_query = input("Write your materials-science query: ")

result = manager.initiate_chat(
    sender=Human,
    recipient=AgentA_Explainer,
    message=initial_query,
)

print("\n--- Chat finished ---")
print(result)




✓ GroupChat and Manager ready.


Write your materials-science query:  materials to extinguish a fire in a forest


chat_manager (to AgentA_Explainer):

materials to extinguish a fire in a forest

--------------------------------------------------------------------------------
AgentA_Explainer (to chat_manager):

***** Suggested tool call (call_8f6YN0tkwzD4Rl9qAXAPJnrD): explain_query_tool *****
Arguments: 
{"query":"materials to extinguish a fire in a forest","query_explanation":"The query is asking for materials or substances that can be used to put out a fire in a forest environment. Key terms include 'extinguish,' which refers to the process of stopping a fire from burning, and 'forest,' which specifies the natural setting where this needs to occur. Secondary elements involve understanding different types of fire-extinguishing materials, such as water, foam, or chemical retardants, and how they are applied in large-scale, outdoor environments such as forests.","context_variables":{}}
***********************************************************************************

----------------------------

Retrieving SummaryDoc documents:   0%|          | 0/103406 [00:00<?, ?it/s]


>>>>>>>> EXECUTED FUNCTION material_retiever...
Call ID: call_vgQ2DneHSs9530pAs1SC1GRM
Input arguments: {'fields': ['material_id', 'formula_pretty', 'band_gap', 'energy_above_hull', 'density'], 'sample_number': 10, 'search_criteria': {'band_gap': {'$gte': 0}, 'energy_above_hull': {'$lte': 0.1}, 'density': {'$gte': 1.0}}, 'context_variables': {'search_criteria': {'band_gap': {'$gte': 0}, 'energy_above_hull': {'$lte': 0.1}, 'density': {'$gte': 1.0}}, 'fields': ['material_id', 'formula_pretty', 'band_gap', 'energy_above_hull', 'density'], 'sample_number': 10, 'mp_results': [MPDataDoc<SummaryDoc>(
formula_pretty='Zr2VNi3',
density=7.7775437455502185,
material_id=MPID(mp-1215561),
energy_above_hull=0.076052419444446,
band_gap=0.0,
fields_not_requested=['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_anonymous', 'chemsys', 'volume', 'density_atomic', 'symmetry', 'property_name', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins',